In [1]:
!pip install torch news-please numpy pandas datasets matplotlib

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_dataset

print(torch.__version__)

print(torch.randn(3,3))
print(torch.cuda.is_available())


c:\Users\ryanh\miniconda3\envs\frame-delta\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.9.1+cpu
tensor([[-1.1174, -0.1549,  1.5280],
        [-0.6363,  0.3967, -1.1734],
        [-0.0835,  1.8568,  0.1832]])
False


## Load the data

In [3]:
# this loads data as an apache arrow format, mapping to disk
data = load_dataset("copenlu/mm-framing")
data_raw = data
data = data['full']
data[1], type(data)

({'uuid': '0000442d-4ce2-4d1c-b654-5561c3cab3b7',
  'title': 'NJ community teams up to help repair historic cemetery damaged by August storms',
  'date_publish': '2023-08-23 22:25:00',
  'source_domain': 'www.cbsnews.com',
  'url': 'https://www.cbsnews.com/philadelphia/video/nj-community-teams-up-to-help-repair-historic-cemetery-damaged-by-august-storms/',
  'political_leaning': 'left_lean',
  'text-topic': 'Historic Cemetery Damage and Restoration',
  'text-topic-exp': "The article discusses the damage caused to a historic cemetery by storms and the community's efforts to repair it. The cemetery is significant as it is the final resting place of several black Civil War veterans.",
  'text-entity-name': "Woodstown church's historic cemetery",
  'text-entity-sentiment': 'Positive',
  'text-entity-sentiment-exp': 'The article describes efforts to repair and clean up the cemetery, indicating a positive sentiment towards the entity.',
  'text-generic-frame': "['Economic', 'Health and safet

### Hydrating the urls test

In [4]:
from newsplease import NewsPlease
test_index = 21000

url = data[test_index]["url"]

article = NewsPlease.from_url(url)

print(article.maintext)
print(url)
data[test_index]

def get_article_text(url):
    text = NewsPlease.from_url(url)
    return(text)



At first, there is some fussing from my 10- and 12-year-old sons. “Will the water be too cold?”, “I don’t want to wear a wetsuit”, “Are there even any decent waves?”… But once their young surf instructor, Josh Mahony, appears with two boards under his arms and the kind of shoulders that suggest he could paddle south to Spain and back in his lunch break, there is only silence as they follow him into the waves.
Some hours later, having caught many waves including their first proper unbroken green ones – before today they’d only surfed more easily managed whitewater – it’s an effort to get them out.
We’ve come to Garretstown in County Cork, a picturesque stretch of sandy beach dotted with smooth grey, brown and purple pebbles. In the car park, an old man is selling honey, eggs and swedes from a picnic table, while all around us is verdant green countryside and cows, save a brutalist concrete toilet block, a yellow airstream pizza van and a wooden sauna sitting above the shoreline.
The boy

After checking several examples of the text, it seems that a large portion of the news are non-political. Seems to be a flaw with the original paper. 

TO DO: Filter the dataset to get just the truly 'political' news stories

TO DO: Check out the other dataset, the 'validation' set to see if it different in terms of its content

In [ ]:
data_raw['valid_framing_subset'][150]

{'uuid': '8cdfd2d3-ff6a-475d-b9e0-baefbc8955b8',
 'title': 'Mets better be right about being ‘capable of better’',
 'date_publish': '2023-06-06 01:15:07',
 'source_domain': 'nypost.com',
 'url': 'https://nypost.com/2023/06/05/mets-better-be-right-about-them-being-capable-of-better/',
 'political_leaning': 'right_lean',
 'text-topic': 'MLB (New York Mets Performance)',
 'text-topic-exp': "The article discusses the performance of the New York Mets in Major League Baseball, their current record, run differential, and individual player performances. It also mentions the team's owner's response to a fan's request to acquire Shohei Ohtani, a current MLB player. The focus is primarily on the Mets' on-field performance and their potential for improvement.",
 'text-entity-name': 'Steve Cohen',
 'text-entity-sentiment': 'neutral',
 'text-entity-sentiment-exp': 'The article portrays Steve Cohen as a businessman focused on the current issues of his team, the Mets, and not actively pursuing other p

So it seems that the validation set is similar in having non-political articles


For now, run with just a subset of the data. Ignore this issue. Parse later before full training runs.

In order to create a hydrated version of the data, bu keep the arrow datasets (which are immutable), then we use a .map() method

In [21]:
# Create a minimal dataset to process in this notebook with hydrated article text

import time
from newsplease import NewsPlease

def scrape_and_add(example):
    """Scrape article text from URL with error handling"""
    url = example['url']

    try:
        # Add a small delay to avoid rate limiting
        time.sleep(0.5)

        # Try to fetch the article
        article = NewsPlease.from_url(url)

        # Return the maintext if available, otherwise empty string
        if article and article.maintext:
            return {"article_text": article.maintext}
        else:
            print(f"No text found for: {url}")
            return {"article_text": ""}

    except Exception as e:
        # Handle 403s and other errors gracefully
        print(f"Error fetching {url}: {type(e).__name__} - {str(e)}")
        return {"article_text": ""}

# Create a small dataset for testing
df = data.shuffle(seed=42).select(range(25))

# Apply the scraping function to each row (batched=False means one at a time)
print("Starting to hydrate URLs...")
df = df.map(scrape_and_add, batched=False)
print("Hydration complete!")

# Check the result
df[3]['article_text']

Starting to hydrate URLs...


Map:  28%|██▊       | 7/25 [00:13<00:30,  1.69s/ examples]

No text found for: https://www.forbes.com/sites/forbesagencycouncil/2024/03/07/understanding-the-power-of-video-for-marketing/


Map:  40%|████      | 10/25 [00:17<00:23,  1.57s/ examples]

No text found for: https://www.axios.com/2024/01/11/microsoft-overtakes-apple-largest-us-company


...

Map:  96%|█████████▌| 24/25 [00:36<00:01,  1.26s/ examples]

No text found for: https://www.forbes.com/sites/johnnavin/2023/05/30/4-cheap-value-stocks-below-book-value-dividend-paying/


Map: 100%|██████████| 25/25 [00:40<00:00,  1.63s/ examples]

Hydration complete!


'A fan leaned over the first base dugout the other day to yell to Mets owner Steve Cohen.\n“Get Ohtani,” said the fan, likely out of desperation.\nCohen answered quickly and good-naturedly, “Who’s he?”\nA few moments later Cohen explained to The Post that he and the Mets are concentrating on more current issues. There are presumably two reasons for this: 1) It helps them stay within baseball rules disallowing tampering with other teams’ stars, and 2) there’s plenty to try to correct right there in front of them.\nLater in that pregame conversation, Cohen noted with a smile — and perhaps slightly sarcastically, at least I hope it was sarcasm — that they were two games over .500. (This was before two more defeats dropped them to an even .500.)\nAnd lest anyone who hasn’t been paying close attention thinks that’s a fluke, please note that they tote a negative-12 run differential. So yes, they are fortunate to be .500.\nThey are also lucky to be in the National League, which is playing lik

## Tokenize and encode

In [27]:
type(df['article_text'])

datasets.arrow_dataset.Column

In [28]:
# use the default auto tokenizer 
from transformers import AutoTokenizer

# Pick a test model checkpoint, change this later if desired
model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model, local_files_only=False)

def tokenize_function(examples):
    return tokenizer(
        examples["article_text"],
        padding="max_length",
        truncation=True
    )

tokenized_df = df.map(tokenize_function, batched=True)
tokenized_df[3]

Map: 100%|██████████| 25/25 [00:00<00:00, 216.92 examples/s]


{'uuid': '8cdfd2d3-ff6a-475d-b9e0-baefbc8955b8',
 'title': 'Mets better be right about being ‘capable of better’',
 'date_publish': '2023-06-06 01:15:07',
 'source_domain': 'nypost.com',
 'url': 'https://nypost.com/2023/06/05/mets-better-be-right-about-them-being-capable-of-better/',
 'political_leaning': 'right_lean',
 'text-topic': 'MLB (New York Mets Performance)',
 'text-topic-exp': "The article discusses the performance of the New York Mets in Major League Baseball, their current record, run differential, and individual player performances. It also mentions the team's owner's response to a fan's request to acquire Shohei Ohtani, a current MLB player. The focus is primarily on the Mets' on-field performance and their potential for improvement.",
 'text-entity-name': 'Steve Cohen',
 'text-entity-sentiment': 'neutral',
 'text-entity-sentiment-exp': 'The article portrays Steve Cohen as a businessman focused on the current issues of his team, the Mets, and not actively pursuing other p

## Set Format

In [33]:
tokenized_df

Dataset({
    features: ['uuid', 'title', 'date_publish', 'source_domain', 'url', 'political_leaning', 'text-topic', 'text-topic-exp', 'text-entity-name', 'text-entity-sentiment', 'text-entity-sentiment-exp', 'text-generic-frame', 'text-generic-frame-exp', 'text-issue-frame', 'text-issue-frame-exp', 'img-generic-frame', 'img-frame-exp', 'img-entity-name', 'img-entity-sentiment', 'img-entity-sentiment-exp', 'gpt-topic', 'article_text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25
})

In [38]:
# set format doesn't delete the other columns, it just hides them. can toggle back to see them
tokenized_df.set_format(
    type="torch",
    columns=['input_ids', 'token_type_ids', 'attention_mask', "text-topic"]
)
tokenized_df.format

{'type': 'torch',
 'format_kwargs': {},
 'columns': ['input_ids', 'token_type_ids', 'attention_mask', 'text-topic'],
 'output_all_columns': False}

In [41]:
# toggling back to see the hidden columns
tokenized_df.reset_format()
tokenized_df.format

{'type': None,
 'format_kwargs': {},
 'columns': ['uuid',
  'title',
  'date_publish',
  'source_domain',
  'url',
  'political_leaning',
  'text-topic',
  'text-topic-exp',
  'text-entity-name',
  'text-entity-sentiment',
  'text-entity-sentiment-exp',
  'text-generic-frame',
  'text-generic-frame-exp',
  'text-issue-frame',
  'text-issue-frame-exp',
  'img-generic-frame',
  'img-frame-exp',
  'img-entity-name',
  'img-entity-sentiment',
  'img-entity-sentiment-exp',
  'gpt-topic',
  'article_text',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'output_all_columns': False}

In [42]:
tokenized_df.set_format(
    type="torch",
    columns=['input_ids', 'token_type_ids', 'attention_mask', "text-topic"]
)

## Creating the data loader

In [ ]:
from torch.utils.data import DataLoader

# Create the conveyor belt
train_dataloader = DataLoader(
    tokenized_df,
    shuffle = True, # mixes the data so the model doesn't learn the order
    batch_size = 4 # number of articles to process at once
)

# in the above, the model sees 16 rows at once, each with 128 numbers


## Run sanity checks

In [ ]:
# We'll just look at the first 2 batches to make sure it works
for i, batch in enumerate(train_dataloader):
    if i >= 2: 
        break
        
    print(f"--- Batch {i} ---")
    print(f"Input IDs shape: {batch['input_ids'].shape}")
    print(f"Attention Mask shape: {batch['attention_mask'].shape}")
    print(f"Labels in this batch: {batch['text-topic']}")
    
    # If you want to see what the model "sees" (first 10 tokens of first row)
    # This turns those IDs back into words!
    # USE tokenizer.decode for specific debugging
    example_text = tokenizer.decode(batch['input_ids'][0][:10])
    print(f"First 10 tokens decoded: {example_text}")
    print("\n")

--- Batch 0 ---
Input IDs shape: torch.Size([4, 512])
Attention Mask shape: torch.Size([4, 512])
Labels in this batch: ['Value Stocks', 'Sports', 'Cookware and Knife Sets', 'Britney Spears']
First 10 tokens decoded: [CLS] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


--- Batch 1 ---
Input IDs shape: torch.Size([4, 512])
Attention Mask shape: torch.Size([4, 512])
Labels in this batch: ['Abortion Pill Access', 'NFL Preseason Game', 'E-Bikes', 'Bank Tax Proposal in Italy']
First 10 tokens decoded: [CLS] danco laboratories, the drugmaker of the


